# 1era Ver. 
#### Rectangulo como contorno del objeto en movimiento

In [10]:
import cv2
#Importar OpenCV

#Toma como entrada el video original y el nombre del archivo de salida
def detect_moving_objects(video_file, output_file):
    cap = cv2.VideoCapture(video_file) #Abre el video original
    
    ####crea el objeto de subtraction del fondo
    #Sustraccion del fondo. Identifica los pixeles que cambiaron en el tiempo. Metodo MOG2
    bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=10, detectShadows=False)
    #varthreshold = sensibilidad al movimiento (al background substraction process) (distancia mahalanobis) 
    #History: cuantas frame toma para crear el backgroung, si se queda quieta en cierta posicion cada history se pierde en el fondo. 
    
    ####Datos para crear el video del output 
    #Ppropiedades del video original
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))#ancho
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))#altura
    fps = int(cap.get(cv2.CAP_PROP_FPS))#frames por segundo
    codec = cv2.VideoWriter_fourcc(*'XVID') #video de salida (XVID) para la compresion del video
    out = cv2.VideoWriter(output_file, codec, fps, (frame_width, frame_height))#mismas propiedades del original
    
    while True:#bucle que procesa cada fotograma
        #captura cada frame
        ret, frame = cap.read() #cuando el fotograma se lee correctamente
        if not ret:#si no se lee bien, termina el bucle
            break
        
        #al fotograma actual le aplica escala de grises (al video)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        ##NOTA: se puede hacer al reves. primero el backsub y luego obtener el frame to frame usando el cap.read()!!!!!!!!!!!!!
        #crea la mascara que resalta los pixeles con respecto al fondo
        fg_mask = bg_subtractor.apply(gray_frame)
        
        #Umbral a la mascara (que esta en escala de grises). Crea imagen binaria donde los pixeles blancos son el objeto de movimiento
        thresh = cv2.threshold(fg_mask, 180, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

        
        #Crea el contorno de las cosas en mmovimmiento (en la imagen binaria)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        #Para cada contorno  detectado
        for contour in contours:
            area = cv2.contourArea(contour) #area del controno
            if 500 < area < 50000:  #para que no detecte ruido(?) (no muy claro)
                x, y, w, h = cv2.boundingRect(contour)#crea el rectangulo alrededor del contorno
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2) #dibuja el rectangulo
        
        #reescribe ese fotograma (marcado) en el video de salida
        out.write(frame)
    
    cap.release()#libera la captura del video
    out.release()


video_file = 'video.mp4'
output_file = 'output_video1.avi'
detect_moving_objects(video_file, output_file)


# 2da Ver. 
#### Contorno filtrado: forma del pez en movimiento
##### Fuente: https://github.com/spmallick/learnopencv/blob/master/Moving-Object-Detection-with-OpenCV/main.py

In [11]:
import cv2

def detect_moving_objects(vid_path, output_file):

    cap = cv2.VideoCapture(vid_path)

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))#ancho
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))#altura
    fps = int(cap.get(cv2.CAP_PROP_FPS))#frames por segundo
    codec = cv2.VideoWriter_fourcc(*'XVID') #video de salida (XVID) para la compresion del video
    out = cv2.VideoWriter(output_file, codec, fps, (frame_width, frame_height))#mismas propiedades del original
    backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=70, detectShadows=False)

    # Check if camera opened successfully
    while True:#bucle que procesa cada fotograma
        #captura cada frame
        ret, frame = cap.read() #cuando el fotograma se lee correctamente
        if not ret:#si no se lee bien, termina el bucle
            break
    
        fg_mask = backSub.apply(frame)


        retval, mask_thresh = cv2.threshold(fg_mask, 180, 255, cv2.THRESH_BINARY)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        mask_eroded = cv2.morphologyEx(mask_thresh, cv2.MORPH_OPEN, kernel)
        contours, hierarchy = cv2.findContours(mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        min_contour_area = 100
        large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
        frame_out = frame.copy()
        for cnt in large_contours:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

        out.write(frame)

    cap.release()
    out.release()



input_video = "video.mp4"
output_file = 'output_video2.avi'
detect_moving_objects(input_video,  output_file)